# Détection des FOG

# I. Chargement des fichiers

In [93]:
import ezc3d
from pyomeca import Analogs, Markers
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt

In [ ]:

# Obtenez le répertoire imus6_subjects7
repertoire_imus6_subjects7 = 'C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/data_article/raw/imus6_subjects7'

# Chargez les données à partir du fichier Excel en utilisant le chemin relatif
pt1_visit_24_tbc_walklr_0_trial_2 = pd.read_excel(os.path.join(repertoire_imus6_subjects7, 'pt1_visit_24_tbc_walklr_0_trial_2.xlsx'))
pt1_visit_24_tbc_walklr_0_trial_4 = pd.read_excel(os.path.join(repertoire_imus6_subjects7, 'pt1_visit_24_tbc_walklr_0_trial_4.xlsx'))

# Supprimez les colonnes contenant des valeurs manquantes pour avoir les colonnes avec les IMUs actifs
pt1_visit_24_tbc_walklr_0_trial_2.dropna(axis=1, inplace=True)
pt1_visit_24_tbc_walklr_0_trial_4.dropna(axis=1, inplace=True)

In [119]:
fichier_brut = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video_overlay_4.c3d"
c3d = ezc3d.c3d(fichier_brut)

labels = c3d['parameters']['ANALOG']['LABELS']['value']
frequence = c3d['parameters']['ANALOG']['RATE']['value'][0]
data = c3d['data']['analogs']

def associer_labels_et_data(labels,data):
    fusion_label_data = {}
    for i, label in enumerate(labels):
        fusion_label_data[label] = data[:, i]
    return fusion_label_data




fusion_label_data = associer_labels_et_data(labels, data)
for label, values in fusion_label_data.items(): 
    parts = label.split('_')
    capteur_nom = parts[1]
    lateralite = parts[0]
    mesure_type = parts[1]
    axes = parts[-1] 




# Fonction json final

In [104]:
import numpy as np
import ezc3d
import json
def filtrer_signaux_utils(c3d):
    labels = c3d['parameters']['ANALOG']['LABELS']['value']
    frequence = c3d['parameters']['ANALOG']['RATE']['value'][0]
    data = c3d['data']['analogs']
    indices = []
    labels_filtre = []
    for i, label in enumerate(labels):
        if 'ACC' in label or 'GYRO' in label:
            indices.append(i)
            labels_filtre.append(label)
    
    data_filtre = data[:, indices]
    data_filtre = data_filtre[0]
    return labels_filtre, data_filtre

def associer_labels_et_data(labels,data):
    fusion_label_data = {}
    for i, label in enumerate(labels):
        fusion_label_data[label] = data[:, i]
    return fusion_label_data

def creer_structure_json(labels, fusion_label_data, patient_id, date_de_naissance, medicaments):
    json_data = {
        "metadata": {
            "Détails du patient": {
                "Identifiant": patient_id,
                "Date de naissance": date_de_naissance,
                "Medicaments": medicaments
            }
        }
    }
    
    capteur_types = set()
    lateralite_types = set()
    mesure_types = set()
    axes_types = set()
    
    # Collecter les différentes étiquettes de la structure
    for label in labels:
        parts = label.split('_')
        lateralite_types.add(parts[0])
        capteur_types.add(parts[1])
        mesure_types.add(parts[2])
        axes_types.add(parts[-1])
    
    # Créer la structure de données
    for capteur in capteur_types:
        capteur_label = {}
        for lateralite in lateralite_types:
            lateralite_label = {}
            for mesure_type in mesure_types:
                mesure_label = {}
                for axes in axes_types:
                    mesure_label[axes] = []
                lateralite_label[mesure_type] = mesure_label
            capteur_label[lateralite] = lateralite_label
        json_data[capteur] = capteur_label
    
    # Remplir la structure de données
    for label, values in fusion_label_data.items():
        parts = label.split('_')
        capteur_nom = parts[1]
        lateralite = parts[0]
        mesure_type = parts[2]
        axes = parts[-1]
        
        json_data[capteur_nom][lateralite][mesure_type][axes] = values.tolist()
    
    return json_data

def creation_json_grace_c3d(file_path, patient_id, date_de_naissance, medicaments, output_path):
    c3d = ezc3d.c3d(file_path)
    labels, data = filtrer_signaux_utils(c3d)
    fusion_label_data = associer_labels_et_data(labels, data)
    
    json_structure = creer_structure_json(labels, fusion_label_data, patient_id, date_de_naissance, medicaments)
    
    with open(output_path, "w") as fichier_json:
        json.dump(json_structure, fichier_json, indent=4)

# Utilisation de la fonction pour traiter le fichier C3D et générer le fichier JSON
fichier_brut = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video_overlay_4.c3d"
patient_id = 1234
date_de_naissance = 45
medicaments = "ON"
chemin_sortie = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/video_overlay_4.json"

creation_json_grace_c3d(fichier_brut, patient_id, date_de_naissance, medicaments, chemin_sortie)


In [105]:
# Charger le fichier JSON
chemin_fichier_json = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/video_overlay_4.json"
with open(chemin_fichier_json, "r") as fichier_json:
    donnees_patient = json.load(fichier_json)

# Accéder aux données spécifiques
accelerometre_right_Z_rectus_femoris = donnees_patient["Rectus Femoris"]["Left"]["GYRO"]["X"]

# Afficher les données
print("Données de l'accéléromètre gauche sur l'axe Z du Rectus Femoris:")
print(accelerometre_right_Z_rectus_femoris)

Données de l'accéléromètre gauche sur l'axe Z du Rectus Femoris:
[0.0126953125, -1.001953125, 0.02587890625, 22.7439022064209, 11.463415145874023, 0.060975611209869385, -0.13427734375, -1.00634765625, -0.03662109375, 18.35365867614746, 13.841464042663574, -1.097561001777649, 0.9150390625, -0.20361328125, 0.2939453125, -8.536585807800293, 19.268293380737305, -2.012195110321045, 0.09033203125, -1.00439453125, 0.09130859375, 20.365854263305664, 7.621951580047607, 2.682926893234253, 0.19189453125, -0.9809570908546448, -0.2153320163488388, -3.2926828861236572, -4.695122241973877, -0.7317073345184326, -0.92333984375, -0.30078125, 0.189453125, 5.609755992889404, 0.060975611209869385, 3.841463565826416, 0.006835937965661287, -1.0009765625, -0.25390625, 7.5, 20.121952056884766, -2.012195110321045]


# II. Filtrage des signaux

In [ ]:
'''from scipy.signal import butter, filtfilt
# Fonction pour concevoir le filtre Butterworth
def butterworth_filter(data, cutoff_frequency, sampling_rate, order=8):
    nyquist_frequency = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist_frequency
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Fréquence de coupure et taux d'échantillonnage
cutoff_frequency = 20  # Hz
sampling_rate = 1 / (pt1_visit_24_tbc_walklr_0_trial_2['time'].iloc[1] - pt1_visit_24_tbc_walklr_0_trial_2['time'].iloc[0])  # Calcul du taux d'échantillonnage

# Colonnes à exclure du filtre
columns_to_exclude = ['time', 'subject_ID', 'freeze_label']

# Colonnes à filtrer
columns_to_filter = [col for col in pt1_visit_24_tbc_walklr_0_trial_2.columns if col not in columns_to_exclude]

# Appliquer le filtre à toutes les colonnes sauf celles exclues
for column in columns_to_filter:
    filtered_data = butterworth_filter(pt1_visit_24_tbc_walklr_0_trial_2[column], cutoff_frequency, sampling_rate)
    pt1_visit_24_tbc_walklr_0_trial_2[f'filtered_{column}'] = filtered_data'''

# III. Fenêtrage de nos données

In [ ]:
import numpy as np

def decouper_en_fenetres(signal, taille_fenetre, decalage):
    """
    Découper le signal en fenêtres temporelles avec un chevauchement de 80%, autrement dit un décalage de 20% entre chaque fenêtre.

    """
    taille_signal = len(signal)
    taille_fenetre_echantillons = int(taille_fenetre * taux_echantillonnage)
    decalage_fenetre = int(decalage * taille_fenetre_echantillons)

    fenetres = []

    debut = 0
    fin = taille_fenetre_echantillons

    while fin <= taille_signal:
        fenetre = signal[debut:fin]
        fenetres.append(fenetre)

        # Mise à jour des indices pour la prochaine fenêtre
        debut = debut + decalage_fenetre
        fin = fin + decalage_fenetre

    return fenetres

# Utilisation de la fonction pour découper le signal en fenêtres
taux_echantillonnage = 148 # en Hz
taille_fenetre = 3  # en secondes
decalage = 0.2  # en %, soit 0.6s


fenetres = decouper_en_fenetres(pt1_visit_24_tbc_walklr_0_trial_2, taille_fenetre, decalage)
    
print("Nombre de fenêtres:", len(fenetres))
print("Taille de chaque fenêtre:", len(fenetres[0]), "frames")
print("Taille du décalage: ", int(decalage * (taille_fenetre * taux_echantillonnage))," frames")

# Pour vérifier si toutes les fenêtres respectent le bon format
for i, fenetre in enumerate(fenetres[0:10]):
    print("Taille de la fenêtre",i+1,":", len(fenetres[i+1]), "frames")
    
'''Attention : Il manque 82 frames dans nos données finales, car nous avons un décalage inférieur à 88 frames pour générer une dernière fenêtre.'''

# III. Visualisation graphique 

In [ ]:
start_time = 0
end_time = 1000

def detect_and_plot_fog_periods(data, start_time, end_time, signals, title):
    # Trouver les indices où le label change de 0 à 1 (début du FOG)
    start_indices = np.where(np.diff(data['freeze_label']) == 1)[0] + 1

    # Trouver les indices où le label change de 1 à 0 (fin du FOG)
    end_indices = np.where(np.diff(data['freeze_label']) == -1)[0] + 1

    # Filtrer les données en fonction des temps spécifiés
    data_time_filtered = data[(data['time'] >= start_time) & (data['time'] <= end_time)]

    # Créer des sous-graphiques pour chaque axe
    fig, axs = plt.subplots(len(signals), figsize=(12, 6 * len(signals)), sharex=True)

    colors = ['royalblue', 'forestgreen', 'darkorange']

    for i, (signal, color) in enumerate(zip(signals, colors)):
        # Tracer les signaux inertiométriques sur chaque sous-graphique
        axs[i].plot(data_time_filtered['time'], data_time_filtered[signal], label="Signal", color=color)

        # Ajouter des lignes verticales pour marquer le début et la fin de chaque période de FOG
        for start, end in zip(start_indices, end_indices):
            axs[i].axvline(data_time_filtered['time'].iloc[start], color='r', linestyle='--', label='Début de FOG')
            axs[i].axvline(data_time_filtered['time'].iloc[end], color='g', linestyle='--', label='Fin de FOG')
        
        # Colorier la zone entre les lignes verticales
            axs[i].fill_between(data_time_filtered['time'].iloc[start:end + 1],
                0, 1, color='gray', alpha=0.5, transform=axs[i].get_xaxis_transform())
        
        # Paramètres des sous-graphiques
        axs[i].set(ylabel='Accélération (m/s)')
        axs[i].set_title(f'{signal}')
        
        # Créer une légende pour chaque sous-graphique
        handles, labels = axs[i].get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        axs[i].legend(by_label.values(), by_label.keys(), loc='upper left')

    # Paramètres communs à tous les sous-graphiques
    fig.suptitle(title)
    axs[-1].set(xlabel='Temps')

    plt.show()

# Application de la fonction

#Tracer les signaux provenant de l'accéléromètre du pied droit
signals_acc_lumbar_to_plot = ['imu_foot_r_ax', 'imu_foot_r_ay', 'imu_foot_r_az']
detect_and_plot_fog_periods(pt1_visit_24_tbc_walklr_0_trial_4, start_time, end_time, signals_acc_lumbar_to_plot, 'Détection des Périodes de FOG des signaux provenant de l\'accéléromètre du pied droit')

#Tracer les signaux provenant du gyroscope du pied droit
signals_gyr_lumbar_to_plot = ['imu_foot_r_gx', 'imu_foot_r_gy', 'imu_foot_r_gz']
detect_and_plot_fog_periods(pt1_visit_24_tbc_walklr_0_trial_4, start_time, end_time, signals_gyr_lumbar_to_plot, 'Détection des Périodes de FOG des signaux provenant du gyroscope du pied droit')

#Tracer les signaux provenant de l'accéléromètre du pied gauche
signals_acc_lumbar_to_plot = ['imu_foot_l_ax', 'imu_foot_l_ay', 'imu_foot_l_az']
detect_and_plot_fog_periods(pt1_visit_24_tbc_walklr_0_trial_4, start_time, end_time, signals_acc_lumbar_to_plot, 'Détection des Périodes de FOG des signaux provenant de l\'accéléromètre du pied droit')

#Tracer les signaux provenant du gyroscope du pied gauche
signals_gyr_lumbar_to_plot = ['imu_foot_l_gx', 'imu_foot_l_gy', 'imu_foot_l_gz']
detect_and_plot_fog_periods(pt1_visit_24_tbc_walklr_0_trial_4, start_time, end_time, signals_gyr_lumbar_to_plot, 'Détection des Périodes de FOG des signaux provenant du gyroscope du pied droit')

# III. Extraction des caractéristiques du signal

# 1. Indice de gel (Moore et al, 2008)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

'''
# Fonction pour calculer l'Indice de Gel (Freezing Index)
def calculate_freezing_index(signal, fs):
    # Paramètres de la méthode de Moore et al. (2008)
    fb_band = (3, 8)  # Bande de gel de la marche en Hz
    lb_band = (0.5, 3)  # Bande de locomotion en Hz
    window_size = int(6 * fs)  # Fenêtre temporelle de 6 secondes

    # Découper le signal en segments de 6 secondes
    num_segments = len(signal) // window_size
    segments = np.array_split(signal, num_segments)

    freezing_index_values = []

    for segment in segments:
        # Calcul de la densité spectrale de puissance (PSD) avec la méthode de Welch
        f, psd = welch(segment, fs=fs, nperseg=window_size)

        # Extraire les indices de fréquence pour les bandes de gel et de locomotion
        fb_indices = np.where((f >= fb_band[0]) & (f <= fb_band[1]))[0]
        lb_indices = np.where((f >= lb_band[0]) & (f <= lb_band[1]))[0]

        # Calculer l'Indice de Gel (Freezing Index)
        freezing_index = np.sum(psd[fb_indices]) / np.sum(psd[lb_indices])
        freezing_index_values.append(freezing_index)

    return np.array(freezing_index_values)

#Application de la fonction
fs = 50  # Fréquence d'échantillonnage en Hz
time = np.arange(0, len(pt1_visit_24_tbc_walklr_0_trial_2['imu_lumbar_ax']), 1) / fs
acceleration_signal = pt1_visit_24_tbc_walklr_0_trial_2['imu_lumbar_ax']

freezing_index_values = calculate_freezing_index(acceleration_signal, fs)

# Tracer l'Indice de Gel sur le temps
plt.figure(figsize=(12, 6))
plt.plot(time[:len(freezing_index_values)], freezing_index_values, label='Indice de Gel (FI)')
plt.xlabel('Temps (s)')
plt.ylabel('Indice de Gel (FI)')
plt.title('Analyse fréquentielle des signaux d\'accéléromètre - Méthode de Moore et al. (2008)')
plt.legend()
plt.grid(True)
plt.show() '''
